In [2]:
import json
from pathlib import Path

path = Path('/Users/jss009/code/therapy-normalization/therapy/data/drugsatfda/drugsatfda_20210817.json')
data = json.load(open(path, 'r'))['results']

In [3]:
from therapy.query import QueryHandler
q = QueryHandler()

***Using Therapy Database Endpoint: http://localhost:8000***


In [4]:
conflict_status = []
no_products = []
valid = []
for r in data:
    try:
        products = r['products']
    except KeyError:
        no_products.append(r)
        continue
    if not all([p['marketing_status'] == products[0]['marketing_status'] for p in products]):
        conflict_status.append(r)
    else:
        valid.append(r)
        
print(f'Total drugs: {len(data)}')
print(f'# with conflicting statuses: {len(conflict_status)}')
print(f'# with no `product` property: {len(no_products)}')

Total drugs: 25017
# with conflicting statuses: 520
# with no `product` property: 274


In [5]:
brand_names = []
for v in valid:
    if 'openfda' in v:
        if 'brand_name' in v['openfda']:
            brand_names += v['openfda']['brand_name']

In [6]:
brand_names = set(brand_names)

In [15]:
print('')
fails = []
for name in brand_names:
    result = q.search_groups(name)
    if result['match_type'] == 0:
        fails.append(name)


In [13]:
print(f'# of failed brand name lookups: {len(fails)}')
print(f'# of unique brand names in drugs@fda: {len(brand_names)}')

# of failed brand name lookups: 2836
# of unique brand names in drugs@fda: 5521


In [14]:
len(data)

25017

In [29]:
key_combos = set()
for d in data:
    key_combos.add(', '.join(sorted(list(d.keys()))))

key_combos

{'application_number, openfda, products, sponsor_name, submissions',
 'application_number, openfda, sponsor_name, submissions',
 'application_number, products, sponsor_name',
 'application_number, products, sponsor_name, submissions',
 'application_number, sponsor_name',
 'application_number, sponsor_name, submissions'}

In [32]:
data[0]['application_number']

'ANDA076165'